In [1]:
%load_ext tensorboard

In [2]:
import PyPDF2
import torch
import tensorflow as tf
import datetime
from transformers import AutoTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_checkpoint = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

dataset_file_name = 'ns_dataset.txt'




2024-10-06 06:55:39.814096: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-06 06:55:39.997193: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-06 06:55:40.066784: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-06 06:55:40.087474: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-06 06:55:40.227015: I tensorflow/core/platform/cpu_feature_guar

In [21]:
import os
from spacy.lang.en import English

nlp = English()
nlp.add_pipe("sentencizer")


# Create dataset
if not os.path.isfile(dataset_file_name):
    path_to_doc_folder = '../docs'
    with open(dataset_file_name, 'w') as dataset_file:
        for root, dirs, files in os.walk(path_to_doc_folder):
            for file_name in files:
                with open(os.path.join(root, file_name), 'rb') as pdf_file:
                    pdf_reader = PyPDF2.PdfReader(pdf_file)
                    for page_num in range(len(pdf_reader.pages)):
                        text = pdf_reader.pages[page_num].extract_text()
                        cleaned_text = text.replace("\n", " ").strip()
                        sentences = list(nlp(cleaned_text).sents)
                        for sentence in sentences:
                            dataset_file.write(str(sentence) + '\n')
                    






In [22]:
from datasets import load_dataset

dataset = load_dataset("text", data_files=dataset_file_name, split='train')  #, "validation": dataset_file_name})
dataset = dataset.train_test_split(test_size=0.3)

dataset['train']['text']

Generating train split: 23995 examples [00:00, 1193816.64 examples/s]


['Suppose that the user terminates the video early at time  At the time of termination, the server stops sending bits (if it hasn’t already sent all the bits in the video).',
 'However, the programming interfaceprovided to your routines—the code that would call your entities from above and from below—is veryclose to what is done in an actual UNIX environment. (',
 'Let t be the timestamp of the ith packet received; let r  be the time at which the ith packet is received.',
 'When there are 10 or fewer simultaneously active users (which happens with probability 0.9996), the aggregate arrival rate of data is less than or equal to 1 Mbps, the output rate of the link.',
 ' Linearit y is nice , becaus e it mean s tha t we can describe n-qubit quantu m gate s by table s that  only describ e the output s produce d whe n the input is on e of the 2n classica l states .',
 'What was your vision for Usenet at the time that you were developing it?',
 'Henningreceived his BS in electrical and indust

In [26]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_datasets['train']['input_ids']

Map (num_proc=4): 100%|██████████| 7199/7199 [00:00<00:00, 34839.05 examples/s]


[[15979,
  577,
  326,
  262,
  2836,
  5651,
  689,
  262,
  2008,
  1903,
  379,
  640,
  220,
  1629,
  262,
  640,
  286,
  19883,
  11,
  262,
  4382,
  9911,
  7216,
  10340,
  357,
  361,
  340,
  5818,
  447,
  247,
  83,
  1541,
  1908,
  477,
  262,
  10340,
  287,
  262,
  2008,
  737],
 [4864,
  11,
  262,
  8300,
  9556,
  330,
  538,
  18657,
  1384,
  284,
  534,
  31878,
  960,
  1169,
  2438,
  326,
  561,
  869,
  534,
  12066,
  422,
  2029,
  290,
  422,
  2174,
  960,
  271,
  845,
  19836,
  284,
  644,
  318,
  1760,
  287,
  281,
  4036,
  4725,
  10426,
  2858,
  13,
  357],
 [5756,
  256,
  307,
  262,
  41033,
  286,
  262,
  340,
  71,
  19638,
  2722,
  26,
  1309,
  374,
  220,
  307,
  262,
  640,
  379,
  543,
  262,
  340,
  71,
  19638,
  318,
  2722,
  13],
 [2215,
  612,
  389,
  838,
  393,
  7380,
  11640,
  4075,
  2985,
  357,
  4758,
  4325,
  351,
  12867,
  657,
  13,
  2079,
  4846,
  828,
  262,
  19406,
  10325,
  2494,
  286,
  1366,
  318

In [31]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

lm_datasets

Map (num_proc=4): 100%|██████████| 7199/7199 [00:00<00:00, 42062.64 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3800
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1596
    })
})

In [32]:
from transformers import AutoModelForCausalLM


model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
model.to('cuda')



GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [33]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [34]:
from transformers import TrainingArguments, Trainer

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2-without_labels",
    eval_strategy = "epoch",
    num_train_epochs=3.0,
    eval_accumulation_steps=1
)


In [35]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,3.845927
2,4.093500,3.759242
3,3.760000,3.736589


TrainOutput(global_step=1425, training_loss=3.8445798211348685, metrics={'train_runtime': 154.7371, 'train_samples_per_second': 73.673, 'train_steps_per_second': 9.209, 'total_flos': 372347869593600.0, 'train_loss': 3.8445798211348685, 'epoch': 3.0})

In [ ]:
trainer.evaluate()